In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [9]:
target_period_hours = 1 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: Sat 10 Feb, 08:00 (22 hours left)
current price: 2468.45
-------------------------------
trading put 2450 and call 2500 (hit ratio: 40%)
cost: 0.011 / value: 0.016, benefit/cost = 1.5x
($) cost: 27.15 / value: 39.50, average gain = 12.35
-------------------------------
trading put 2425 and call 2525 (hit ratio: 33%)
cost: 0.006 / value: 0.011, benefit/cost = 1.9x
($) cost: 14.07 / value: 26.56, average gain = 12.49
-------------------------------
trading put 2400 and call 2550 (hit ratio: 24%)
cost: 0.003 / value: 0.007, benefit/cost = 2.3x
($) cost: 7.90 / value: 18.03, average gain = 10.13
-------------------------------
trading put 2375 and call 2575 (hit ratio: 17%)
cost: 0.002 / value: 0.005, benefit/cost = 2.6x
($) cost: 4.69 / value: 12.34, average gain = 7.65


[{'put': 2425.0, 'call': 2525.0, 'quantity': 1},
 {'put': 2400.0, 'call': 2550.0, 'quantity': 2}]

In [10]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
2375.0,100.330391,NaN,0.0310,0.0460,6.815877,0.002761,0.0005,0.0008
2400.0,78.369237,NaN,0.0280,0.0320,9.854723,0.003992,0.0010,0.0014
2425.0,57.988216,NaN,0.0190,0.0235,14.473702,0.005863,0.0022,0.0027
2450.0,40.171272,NaN,0.0130,0.0140,21.656757,0.008773,0.0049,0.0055
2475.0,26.552215,0.010757,0.0080,0.0090,33.037701,NaN,0.0095,0.0105
2500.0,17.841495,0.007228,0.0050,0.0055,49.326980,NaN,0.0140,0.0195
2525.0,12.084297,0.004896,0.0026,0.0030,68.569783,NaN,0.0055,NaN
2550.0,8.178931,0.003313,0.0014,0.0018,89.664416,NaN,0.0150,NaN
2575.0,5.523093,0.002237,0.0008,0.0011,112.008579,NaN,0.0255,NaN


In [11]:
#print(simulation.evaluate_put(2300), simulation.get_put_bid(2300))
print(simulation.evaluate_call(2400), simulation.get_call_bid(2400))

78.36923724510132 69.11659999999999


# Sanity checks

In [7]:
df = trading_model.valuation
df.groupby([df.index.year, df.index.month]).size() / 24

dateTime  dateTime
2022      1           31.000000
          2           28.000000
          3           31.000000
          4           30.000000
          5           31.000000
          6           30.000000
          7           31.000000
          8           31.000000
          9           30.000000
          10          31.000000
          11          30.000000
          12          31.000000
2023      1           31.000000
          2           28.000000
          3           30.958333
          4           30.000000
          5           31.000000
          6           30.000000
          7           31.000000
          8           31.000000
          9           30.000000
          10          31.000000
          11          30.000000
          12          31.000000
2024      1           30.041667
dtype: float64